## Maximum path sum I

Problem 18

By starting at the top of the triangle below and moving to adjacent numbers on the row below, the maximum total from top to bottom is 23.

3
7 4
2 4 6
8 5 9 3

That is, 3 + 7 + 4 + 9 = 23.

Find the maximum total from top to bottom of the triangle below:

75  
95 64  
17 47 82  
18 35 87 10  
20 04 82 47 65  
19 01 23 75 03 34  
88 02 77 73 07 63 67  
99 65 04 28 06 16 70 92  
41 41 26 56 83 40 80 70 33  
41 48 72 33 47 32 37 16 94 29  
53 71 44 65 25 43 91 52 97 51 14  
70 11 33 28 77 73 17 78 39 68 17 57  
91 71 52 38 17 14 91 43 58 50 27 29 48  
63 66 04 68 89 53 67 30 73 16 69 87 40 31  
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23  

NOTE: As there are only 16384 routes, it is possible to solve this problem by trying every route. However, Problem 67, is the same challenge with a triangle containing one-hundred rows; it cannot be solved by brute force, and requires a clever method! ;o)


Link: https://projecteuler.net/problem=18

Date solved:  
2022/03/08

In [1]:
# imports
import sys
sys.path.append('../../modules')

import parsing
import graphs

In [19]:
class Node:
    
    def __init__(self,value=None,flag=None):
        self.value = value
        self.flag = flag

    def __repr__(self):
        return str(self.value)

class Edge:

    def __init__(self,length = 0,n1 = None,n2 = None):
        self.length = length
        self.n1 = n1
        self.n2 = n2

    def __repr__(self):
        return f'{self.n1} -{self.length if self.length else ""}> {self.n2}'

    def has_node(self,node):
        return node in (self.n1,self.n2)
    
    def get_nodes(self):
        return (self.n1,self.n2)


class Graph:

    def __init__(self,edges = None):
        self.edges = edges if edges else []
        self.nodes = self.get_nodes()

    def __repr__(self):
        output = ''
        for edge in self.edges:
            output += f'{edge}\n'
        return output

    def add_edge(self,edge):
        self.edges.append(edge)
        self.nodes = self.get_nodes()

    def get_nodes(self):
        nodes = []
        for edge in self.edges:
            if edge.n1 not in nodes:
                nodes.append(edge.n1)
            if edge.n2 not in nodes:
                nodes.append(edge.n2)
        return nodes


    def has_node(self,node):
        return node in self.nodes

    def has_nodes(self,node_list):
        return all([self.has_node(node) for node in node_list])

        
        

    def get_adjacent_edges(self,node) -> set:
        edges = set()
        for edge in self.edges:
            if edge.has_node(node):
                edges.add(edge)
        return edges

    def get_adjacent_nodes(self,node) -> set:
        nodes = set()
        for edge in self.get_adjacent_edges(node):
            n1,n2 = edge.get_nodes()
            nodes.add(n1)
            nodes.add(n2)
        nodes.remove(node)
        return nodes

    def mark_node(self,node,flag_name):
        node.flag = flag_name

    def mark_nodes(self,node_set,flag_name):
        for node in node_set:
            self.mark_node(node,flag_name)

    def reset_node(self,node):
        self.make_node(node,None)

    def reset_nodes(self,node_set):
        self.make_nodes(node,None)

    def get_adjacent_unexplored_nodes(self,node):
        return set([n for n in self.get_adjacent_nodes(node) if n.flag!='EXPLORED'])

    def get_all_connected_nodes(self,node):
        def explore_deep(node,node_set):
            to_explore = self.get_adjacent_unexplored_nodes(node)
            self.mark_nodes(to_explore,'EXPLORED')
            node_set.add(node)
            for new_node in to_explore:
                node_set = explore_deep(new_node,node_set)
            return node_set
        all_connected_nodes = explore_deep(node,set([node]))
        yield all_connected_nodes
        self.reset_nodes(all_connected_nodes)


    def path_exists(self,n0,n1):
        return n0 in self.get_all_connected_nodes(n1)


    def shortest_path_directional(self,n0,n1):
        if not self.path_exists(n0,n1):
            raise Exception('NO PATH EXISTS')
        connected_nodes = self.get_all_connected_nodes(n0)
        self.mark_nodes(connected_nodes,'UNSEEN')
        self.mark_node(n0,'EXPLORED')
        adjacent_nodes = self.get_adjacent_nodes(n0)
        self.mark_nodes(adjacent_nodes,'FRINGE')
        





def nodify(array)-> list:
    node_array = [[Node(val) for val in row] for row in array]
    return node_array

In [21]:
n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
n4 = Node(4)
n5 = Node(5)
n6 = Node(6)
e1 = Edge(11,n1,n2)
e2 = Edge(12,n1,n3)
e3 = Edge(13,n3,n4)
e4 = Edge(14,n4,n1)
e5 = Edge(15,n5,n6)
e6 = Edge(16,n6,n5)
e7 = Edge(17,n1,n1)

graph = Graph([e1,e2,e3,e4,e5,e6,e7])
graph.path_exists(n1,n6)

False

In [7]:
# %%timeit # results = 77.2 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


triangle = '''75  
95 64  
17 47 82  
18 35 87 10  
20 04 82 47 65  
19 01 23 75 03 34  
88 02 77 73 07 63 67  
99 65 04 28 06 16 70 92  
41 41 26 56 83 40 80 70 33  
41 48 72 33 47 32 37 16 94 29  
53 71 44 65 25 43 91 52 97 51 14  
70 11 33 28 77 73 17 78 39 68 17 57  
91 71 52 38 17 14 91 43 58 50 27 29 48  
63 66 04 68 89 53 67 30 73 16 69 87 40 31  
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23  '''


array = parsing.read_array_str(triangle)

node_array = graphs.nodify(array)

graph = graphs.Graph()

for i,row in enumerate(node_array):
    for j,node in enumerate(row):
        if i > 0:
            if j > 0:
                above_node = node_array[i-1][j-1]
                edge0 = graphs.Edge(0,above_node,node)
                graph.add_edge(edge0)
            if j < i:
                above_node = node_array[i-1][j]
                edge1 = graphs.Edge(0,above_node,node)
                graph.add_edge(edge1)

graph


        



75 -> 95
75 -> 64
95 -> 17
95 -> 47
64 -> 47
64 -> 82
17 -> 18
17 -> 35
47 -> 35
47 -> 87
82 -> 87
82 -> 10
18 -> 20
18 -> 4
35 -> 4
35 -> 82
87 -> 82
87 -> 47
10 -> 47
10 -> 65
20 -> 19
20 -> 1
4 -> 1
4 -> 23
82 -> 23
82 -> 75
47 -> 75
47 -> 3
65 -> 3
65 -> 34
19 -> 88
19 -> 2
1 -> 2
1 -> 77
23 -> 77
23 -> 73
75 -> 73
75 -> 7
3 -> 7
3 -> 63
34 -> 63
34 -> 67
88 -> 99
88 -> 65
2 -> 65
2 -> 4
77 -> 4
77 -> 28
73 -> 28
73 -> 6
7 -> 6
7 -> 16
63 -> 16
63 -> 70
67 -> 70
67 -> 92
99 -> 41
99 -> 41
65 -> 41
65 -> 26
4 -> 26
4 -> 56
28 -> 56
28 -> 83
6 -> 83
6 -> 40
16 -> 40
16 -> 80
70 -> 80
70 -> 70
92 -> 70
92 -> 33
41 -> 41
41 -> 48
41 -> 48
41 -> 72
26 -> 72
26 -> 33
56 -> 33
56 -> 47
83 -> 47
83 -> 32
40 -> 32
40 -> 37
80 -> 37
80 -> 16
70 -> 16
70 -> 94
33 -> 94
33 -> 29
41 -> 53
41 -> 71
48 -> 71
48 -> 44
72 -> 44
72 -> 65
33 -> 65
33 -> 25
47 -> 25
47 -> 43
32 -> 43
32 -> 91
37 -> 91
37 -> 52
16 -> 52
16 -> 97
94 -> 97
94 -> 51
29 -> 51
29 -> 14
53 -> 70
53 -> 11
71 -> 11
71 -> 33
44

## ANSWER = 142913828922